In [11]:
import cv2
import mediapipe as mp

In [12]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [13]:
cap = cv2.VideoCapture(0)  # 0 for webcam, or provide the video file path

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    imgrgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(imgrgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                x = int(lm.x * frame.shape[1])
                y = int(lm.y * frame.shape[0])
                cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)

            # Check for an open hand gesture
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            if index_finger_tip.y < thumb_tip.y:  
                cv2.putText(frame, 'Hand Is Open', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            
             # Check for a "like" gesture
            thumb = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            index = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            middle = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
            ring = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
            pinky = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]
            
            # Define the criteria for a "like" gesture (thumb near index finger)
            thumb_index_dist = ((thumb.x - index.x) ** 2 + (thumb.y - index.y) ** 2) ** 0.5
            if thumb_index_dist < 0.05:  # Adjust this threshold based on your testing
                cv2.putText(frame, 'Like Gesture Detected', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                
    out.write(frame)

    cv2.imshow("Frame", frame)

    # Check for 'q' key press to close the camera
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release VideoCapture and VideoWriter objects
cap.release()
out.release()
cv2.destroyAllWindows()

